In [16]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (131 kB/s)
Reading package lists... Done


In [17]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2020-11-03 22:27:48--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2020-11-03 22:27:49 (9.42 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [19]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [20]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

### Create DataFrames to match tables

In [21]:
import pandas as pd

In [22]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
reviewdf= df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", "yyyy-mm-dd")])


In [ ]:
reviewdf.show()

+--------------+-----------+----------+--------------+------------------------------------+
|     review_id|customer_id|product_id|product_parent|to_date(`review_date`, 'yyyy-mm-dd')|
+--------------+-----------+----------+--------------+------------------------------------+
|R3W4P9UBGNGH1U|   16414143|B00YL0EKWE|     852431543|                          2015-01-31|
|R15V54KBMTQWAY|   50800750|B00XK95RPQ|     516894650|                          2015-01-31|
| RY8I449HNXSVF|   15184378|B00SXRXUKO|     984297154|                          2015-01-31|
|R18TLJYCKJFLSR|   10203548|B009V5X1CE|     279912704|                          2015-01-31|
|R1NK26SWS53B8Q|     488280|B00D93OVF0|     662791300|                          2015-01-31|
|R11LOHEDYJALTN|   13334021|B00XVGJMDQ|     421688488|                          2015-01-31|
|R3ALQVQB2P9LA7|   27520697|B00KQW1X1C|     554285554|                          2015-01-31|
|R3MWLXLNO21PDQ|   48086021|B00IP1MQNK|     488006702|                          

In [23]:
# Create the customers_table DataFrame
#customers_df = df.groupby("customer_id").agg({""}).withColumnRenamed("", "customer_count")
customers_df = df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")

In [24]:
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   46909180|             6|
|   42560427|             7|
|   43789873|             3|
|   22037526|             2|
|   34220092|             2|
|   42801586|             1|
|    9565734|             2|
|   15829398|             1|
|   38247118|             1|
|   32478248|             2|
|   48114630|             1|
|   23085063|             1|
|   32787070|             3|
|   43515569|             1|
|    4919528|             2|
|    5088547|             2|
|   41852407|             3|
|   49703087|             1|
|   12713799|             1|
|   36728141|             8|
+-----------+--------------+
only showing top 20 rows



In [25]:
# Create the products_table DataFrame and drop duplicates. 
#products_df = df.select([]).drop_duplicates()
products_df = df.select(["product_id", "product_title"]).drop_duplicates()

In [26]:
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B010LVPJH6|LG G Watch Urbane...|
|B00K5ZNXZ4|Minisuit Sporty A...|
|B00S9RBQOK|iPhone 6 Plus Cas...|
|B0116N7GYC|Selfie Stick, Por...|
|B009UNH0CY|Plantronics Voyag...|
|B00L8GFYAG|Eallc New Quality...|
|B00R3LMTI0|Kaleidio [Wallop ...|
|B011R0VG36|Galaxy Note 4 Cas...|
|B00BXX0QVQ|iKross Black Dual...|
|B00F4AYI2M|Incipio DualPro C...|
|B00GPI3OHC|Retevis H-777 2 W...|
|B00Y9ZUVU6|Tiwkich 2 in 1 Du...|
|B00W65SYHS|LG G4 case, Caseo...|
|B00V5OU6CW|S5 Leather case,P...|
|B00LP3FSH6|Escort Coiled Sma...|
|B00MIO2KRC|Black Box G1W-C C...|
|B00V5FZM0M|   KoKo Cases 5/5S !|
|B00T1KO2TA|iPhone 6 & 6S Cas...|
|B00YU9XOTQ|Galaxy S5 Screen ...|
|B00PI7IGHE|Soyan Latest DZ09...|
+----------+--------------------+
only showing top 20 rows



In [27]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
#review_id_df = df.select([, to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df = df.select([to_date("review_date", 'yyyy-MM-dd').alias("review_date")])

In [28]:
review_df= df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", "yyyy-mm-dd").alias("review_date")])

In [29]:
reviewdf.show()

+--------------+-----------+----------+--------------+------------------------------------+
|     review_id|customer_id|product_id|product_parent|to_date(`review_date`, 'yyyy-mm-dd')|
+--------------+-----------+----------+--------------+------------------------------------+
|R3W4P9UBGNGH1U|   16414143|B00YL0EKWE|     852431543|                          2015-01-31|
|R15V54KBMTQWAY|   50800750|B00XK95RPQ|     516894650|                          2015-01-31|
| RY8I449HNXSVF|   15184378|B00SXRXUKO|     984297154|                          2015-01-31|
|R18TLJYCKJFLSR|   10203548|B009V5X1CE|     279912704|                          2015-01-31|
|R1NK26SWS53B8Q|     488280|B00D93OVF0|     662791300|                          2015-01-31|
|R11LOHEDYJALTN|   13334021|B00XVGJMDQ|     421688488|                          2015-01-31|
|R3ALQVQB2P9LA7|   27520697|B00KQW1X1C|     554285554|                          2015-01-31|
|R3MWLXLNO21PDQ|   48086021|B00IP1MQNK|     488006702|                          

In [30]:
# Create the vine_table. DataFrame
# vine_df = df.select([])
#vine_df = grocery_df.select(['review_id','star_rating','helpful_votes','total_votes,'vine','verified_purchase'])

# Create the vine_table. DataFrame
# vine_df = df.select([])
vine_df = df.select(['review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3W4P9UBGNGH1U|          2|            1|          3|   N|                Y|
|R15V54KBMTQWAY|          4|            0|          0|   N|                N|
| RY8I449HNXSVF|          5|            0|          0|   N|                Y|
|R18TLJYCKJFLSR|          5|            0|          0|   N|                Y|
|R1NK26SWS53B8Q|          5|            0|          0|   N|                Y|
|R11LOHEDYJALTN|          5|            0|          0|   N|                Y|
|R3ALQVQB2P9LA7|          4|            0|          0|   N|                Y|
|R3MWLXLNO21PDQ|          5|            0|          0|   N|                Y|
|R2L15IS24CX0LI|          5|            0|          0|   N|                Y|
|R1DJ8976WPWVZU|          3|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [31]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://mod16chall.cctkcrlndac5.us-east-2.rds.amazonaws.com:5432/Mod16ChalDel1"
config = {"user":"postgres", 
          "password": "328Crazy", 
          "driver":"org.postgresql.Driver"}

In [36]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

Py4JJavaError: ignored

In [35]:
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

Py4JJavaError: ignored

In [33]:
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

In [40]:
# Write products_df to table in RDS
# about 3 min

products_df.write.jdbc(url=jdbc_url,table='products_table', mode=mode, properties=config)

Py4JJavaError: ignored

In [38]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

Py4JJavaError: ignored

In [ ]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:43099)
Traceback (most recent call last):
  File "/content/spark-3.0.1-bin-hadoop2.7/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/content/spark-3.0.1-bin-hadoop2.7/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: ignored

In [ ]:
#mod16df = df.toPandas()

Py4JJavaError: ignored

In [46]:
#mod16df[mod16df["total_votes" >= 20]]

NameError: ignored

In [ ]:
# Deliverable 2

In [42]:
#data is filtered to create a DataFrame or table where there are 20 or more total votes
filtered_votes_df = df.filter(['total_votes'] >= 20)

TypeError: ignored

In [43]:
#data is filtered to create a DataFrame or table where the percentage of helpful_votes is equal to or greater than 50%
helpful_votes_df = df.filter(['helpful_votes'] >=50%)

SyntaxError: ignored

In [44]:
#data is filtered to create a DataFrame or table where there is a Vine review
vine_review_df = df.filter(['vine'] = Y)

SyntaxError: ignored

In [45]:
# data is filtered to create a DataFrame or table where there isn’t a Vine review
no_vine_review_df = df.filter(['vine'] = N)

SyntaxError: ignored

In [ ]:
#How many Vine reviews 
#customers_df = df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
#vine_df = df.select(['review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase'])
vine_reviews = vine_df('review_id').agg({'review_id': 'count'})

In [ ]:
#non-Vine reviews
non_vine_reviews = df.select(['vine'] == N).count()


In [ ]:
#How many Vine reviews were 5 stars? 
vine_five_star_rating = vine_df.select (['star_rating'] == 5).count()

In [ ]:
#How many non-Vine reviews were 5 stars?
five_star_non_vine_reviews = df.select ((('vine')==N) == 5).count()

In [ ]:
#What percentage of Vine reviews were 5 stars?

In [ ]:
#What percentage of non-Vine reviews were 5 stars?